In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import StaleElementReferenceException
import time
import re
import pandas as pd


In [1]:
class Kipris() :
    def __init__(self):
        # 상태 - status
        # 제목 - title
        # 출원번호 - application_number
        # 출원일자 - application_date
        # 출원인 - applicant
        # 대리인 - representative
        # 최종권리자 - final_owner
        # 우선권주장일자 - priority_date
        # 국제출원일자 - international_filing_date
        # 요약 - summary

        self.driver = webdriver.Chrome()
        self.m_page_code = ''   # 메인 페이지 코드
        self.df = pd.DataFrame(columns=['status', 'title', 'ap_num', 'application_date', 'applicant','representative',
                            'final_owner', 'priority_date', 'international_filing_date', 'summary'])
        
        self.login_id = ''   # 로그인 ID
        self.login_pw = ''   # 로그인 PW
        
        self.queryText = '("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"딥러닝"+"deep learning")*AD=[20130624~20240624]'  # 검색어

    def open(self) :
        # 사이트 열기
        url = 'http://www.kipris.or.kr/khome/main.jsp'

        self.driver.get(url)
        
        # 처음에 열린 페이지들 코드 (메인페이지, 팝업페이지)
        page_dict = {idx: self.driver.window_handles[idx] for idx in range(len(self.driver.window_handles))}

        # 메인페이지 코드
        self.m_page_code = page_dict[0]

    def close(self):
        # 브라우저 종료
        self.driver.quit()

    # 로그인 
    def login(self) :
        print('\nlogin() start')

        # 로그인 버튼 태그
        login = self.driver.find_element(By.ID, 'btnLogin')
        login.click()

        # 로그인 팝업창 선택
        self.driver.switch_to.window(self.driver.window_handles[1])

        # 로그인 ID
        login_id_tag = self.driver.find_element(By.ID, 'login_id')
        login_id_tag.send_keys(self.login_id)

        # 로그인 pw
        login_pw_tag = self.driver.find_element(By.ID, 'login_pw')
        login_pw_tag.send_keys(self.login_pw)

        # 로그인 버튼
        btn_login = self.driver.find_element(By.ID, 'btn_login')
        btn_login.click()

        # 5초 기다린 후 팝업 닫기
        time.sleep(5)
        self.pop_close()

        print('login() end')

    # 팝업닫기
    def pop_close(self) :
        print('\npop_close() start')

        # 팝업이 1개이상 있는지 최대 10초동안 확인
        wait = WebDriverWait(self.driver, 10)
        wait.until(lambda d: len(self.driver.window_handles) > 1)

        print('페이지 갯수 : ' , len(self.driver.window_handles))

        # 현재선택되어 있는 페이지를 메인페이지로 변경
        self.driver.switch_to.window(self.m_page_code)

        # 팝업이 있을 경우 팝업 닫기
        if len(self.driver.window_handles) > 1 :
            total_site = self.driver.window_handles # 열린 사이트 갯수 (메인페이지 + 팝업창)

            print(f'총 페이지 : {total_site}')
            print(f'메인 페이지 코드 : {self.m_page_code}')

            for page_code in total_site :
                # 열린 사이트들에서 메인 페이지를 제외한 창 닫기)
                if page_code != self.m_page_code :
                    self.driver.switch_to.window(page_code) # 현재 선택된 창을 팝업창으로 변경
                    self.driver.close()

            # 팝업창을 다 닫은 후 메인페이지를 다시 선택
            self.driver.switch_to.window(self.m_page_code)
            print('if end')

        print('pop_close() end')

    # 페이지 이동
    def patent_move(self) :
        print('patent_move start')

        # 페이지이동 위한 특허·실용신안 태그 가져오기
        patent_tag = WebDriverWait(self.driver,3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,'#gnb ul.cbp-hrsub-inner.cbp-hrsub-inner.depth2 li:nth-child(1) a'))
        )

        # 클릭하여 페이지 이동
        patent_tag.click()

        # 페이지 이동 잠시 대기
        time.sleep(3)

        # 검색창 입력
        input_tag = self.driver.find_element(By.ID,'queryText')
        input_tag.send_keys(self.queryText)
        input_tag.send_keys(Keys.ENTER)

        time.sleep(3)

        # 우선권주장일자, 국제출원일자 정보 보기 위한 리스트 형식 변경 (요약 함께보기)
        btnTextView_tag = self.driver.find_element(By.CSS_SELECTOR, '#btnTextView')
        btnTextView_tag.click()

        time.sleep(3)
        
        # 페이지 목록 갯수 설정
        select = Select(self.driver.find_element(By.ID,'opt28'))
        # select태그의 옵션값 90 선택
        select.select_by_value('90')

        time.sleep(3)

        # 설정된 페이지 목록 갯수로 다시 조회
        select_a = self.driver.find_element(By.CSS_SELECTOR,'#pageSel a')
        select_a.click() # 'go' 버튼 클릭
            
        print('patent_move end')
    
    # 데이터 셋팅
    def data_setting(self) :
        # 상태 제목 출원번호 출원일자 출원인 대리인 최종권리자 우선권주장일자 국제출원일자 요약
        # 상태 - status
        # 제목 - title
        # 출원번호 - application_number
        # 출원일자 - application_date
        # 출원인 - applicant
        # 대리인 - representative
        # 최종권리자 - final_owner
        # 우선권주장일자 - priority_date
        # 국제출원일자 - international_filing_date
        # 요약 - summary

        print('\ndata_setting start')

        articles_info = self.driver.find_element(By.CSS_SELECTOR, 'p.articles').text # 현재 페이지번호 / 총페이지 번호 태그
        total_page = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip()) # 총 페이지

        current_page_tag = self.driver.find_element(By.CSS_SELECTOR, '#divMainArticle div.float_left span.current') # 현재 페이지

        current_page = re.sub(r'[^\d]','',current_page_tag.text)

        # 현재 페이지 번호가 총페이지 번호가 될때까지
        while int(current_page) != (int(total_page) + 1) :
            print('while = current_page ' , current_page)
            data = self.driver.execute_script("""
                    //  전체목록 메인 div 태그
                    var main_section = document.querySelector('#divMainArticle section.search_section');

                    //  특허별 article 태그
                    var divs = main_section.querySelectorAll('[id*="divViewSel"]'); //  id* -> 해당 문자가 포함되어 있는 모든 ID
                    
                    var status_list = []                    //  상태
                    var title_list = []                     //  제목
                    var ap_num_list = []                    //  출원번호
                    var application_date_list = []          //  출원일자
                    var applicant_list = []                 //  출원인
                    var representative_list = []            //  대리인
                    var final_owner_list = []               //  최종권리자
                    var priority_date_list = []             //  우선권주장일자
                    var international_filing_date_list = [] //  국제출원일자
                    var summary_list = []                   //  요약
                    
                    //  현재 페이지에 조회된 목록 갯수 만큼 돌리기
                    for(i=0; i<divs.length; i++){
                        div = divs[i];
                        
                        //  타이틀
                        var h1_tag = div.querySelector('h1[class*="stitle"]');
                        
                        //  등록상태
                        var statusEl = h1_tag.querySelector('#iconStatus');
                        var status = statusEl ? statusEl.textContent: '';   //  해당 테그가 없을 경우 빈값으로 표시
                        status_list.push(status);

                        console.log('status  ' + status);

                        //  제목
                        var titleEl = h1_tag.querySelector('a:nth-of-type(2)');
                        var title = titleEl ? titleEl.textContent : '';
                        title_list.push(title);

                        console.log('title  ' + title);

                        //  출원번호 출원일자 출원인 대리인 최종권리자 DIV
                        var divRealContent = div.querySelector('#divRealContent' + i);

                        //  출원번호
                        var ap_num_el = divRealContent.querySelector('li:nth-child(2) span.point01');
                        var ap_num = ap_num_el ? ap_num_el.textContent : '';
                        ap_num_list.push(ap_num);

                        console.log('출원번호 :   ' + ap_num);
                                                    
                        //  출원일자
                        var ap_date_el = divRealContent.querySelector('li:nth-child(3)');
                        var ap_date = ap_date_el ? ap_date_el.textContent.replace(/[^\d]/g,'') : '';    // 출원일자 : 2020.09.25 -> 숫자만 추출
                        application_date_list.push(ap_date);

                        console.log('application_date_list :   ' + ap_date);
                                                    
                        //  출원인
                        var applicant_el = divRealContent.querySelector('ul.search_info_list li:nth-child(4) a');
                        var applicant = applicant_el ? applicant_el.textContent : '';
                        applicant_list.push(applicant);

                        console.log('applicant :   ' + applicant);

                        //  대리인
                        var representative_el= divRealContent.querySelector('ul.search_info_list li:nth-child(10) a');
                        var representative = representative_el ? representative_el.textContent : '';
                        representative_list.push(representative);

                        console.log('representative :   ' + representative);
                                                    
                        //  최종권리자
                        var final_owner_el = divRealContent.querySelector('ul.search_info_list li:nth-child(11) a');
                        var final_owner = final_owner_el ? final_owner_el.textContent : '';
                        final_owner_list.push(final_owner);

                        console.log('final_owner :   ' + final_owner);
                                                    
                        //  우선권주장일자 국제출원일자 DIV
                        var div_search_detail = divRealContent.querySelector('#div_search_detail_' + ap_num);
                                                    
                        //  우선권주장일자
                        var priority_date_el = div_search_detail.querySelector('ul li:nth-child(1)');
                        var priority_date = priority_date_el ? priority_date_el.textContent.replace(/[^\d]/g,'') : '';
                        priority_date_list.push(priority_date);

                        console.log('priority_date :   ' + priority_date);
                                                    
                        //  국제출원일자
                        var international_filing_date_el = div_search_detail.querySelector('ul li:nth-child(2)');
                        var international_filing_date = international_filing_date_el ? international_filing_date_el.textContent.replace(/[^\d]/g,'') : '';
                        international_filing_date_list.push(international_filing_date);

                        console.log('international_filing_date :   ' + international_filing_date);

                        var summary_el = div_search_detail.querySelector('div.search_txt');
                        var summary = summary_el ? summary_el.textContent.replace(/[\\n\\t]+/g,'') : '';
                        summary_list.push(summary);

                        console.log('summary :   ' + summary);
                    }

                    return {
                        'status'                    :   status_list,
                        'title'                     :   title_list,
                        'ap_num'                    :   ap_num_list,
                        'application_date'          :   application_date_list,
                        'applicant'                 :   applicant_list,
                        'representative'            :   representative_list,
                        'final_owner'               :   final_owner_list,
                        'priority_date'             :   priority_date_list,
                        'international_filing_date' :   international_filing_date_list,
                        'summary' :   summary_list                                    
                    };
                    
                """)

            # 데이터프레임 셋팅
            new_df = pd.DataFrame({
                    'status':data['status'],
                    'title':data['title'],
                    'ap_num':data['ap_num'],
                    'application_date':data['application_date'],
                    'applicant':data['applicant'],
                    'representative':data['representative'],
                    'final_owner':data['final_owner'],
                    'priority_date':data['priority_date'],
                    'international_filing_date':data['international_filing_date'],
                    'summary':data['summary']
                })
                
            # 합치기 전 df 데이터 갯수
            dfLength = len(self.df)

            self.df = pd.concat([self.df, new_df], ignore_index=True)

            # 합치기 전 df 데이터 갯수와 합친 후의 df 데이터 갯수를 비교해 같아질때까지 1초씩 대기
            while len(self.df) != (len(new_df) + dfLength) :
                time.sleep(1)
                
            
            page = int(current_page) + 1
            print(f'page : {page}')
            self.driver.execute_script("""SetPageAjax(arguments[0])""" , page) # 해당 사이트의 목록 번호에 있는 페이지이동 함수 직접호출해서 해당 페이지로 이동


            next_page = int(current_page) + 1   # 다음페이지 번호

            # 페이지 이동 
            while True:
                # 한번씩 current_page 값을 가져오지 못하는 경우가 있어 에러 날 경우 다시 진행
                current_page = ""
                try :
                    current_page_element = WebDriverWait(self.driver,10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR,'#divMainArticle div.float_left span.current'))
                    )

                    # 1000페이지가 넘어가게되면 1,001로 표시되 숫자만 추출
                    current_page = re.sub(r'[^\d]','',current_page_element.text)

                    current_page = int(current_page)
                except StaleElementReferenceException as se:
                    print(se)
                    continue

                
                print(f'current_page : {current_page} , total_page  : {total_page}')

                # 현재페이지가 마지막페이지가 아닐 경우 실행
                if current_page != (total_page+1):

                    # 현재페이지가 다음페이지가 아닐경우 1초씩 대기
                    if next_page != current_page:
                        time.sleep(1)
                        print(f'next_page : {next_page}, current_page : {current_page}')
                    else:
                        break
                else :
                    break
        print('data_setting end\n')

    def start(self) :
        # 페이지 오픈
        self.open()

        # 팝업 닫기
        self.pop_close()

        # 로그인 진행
        self.login()

        # 페이지 이동
        self.patent_move()

        time.sleep(10)

        # 데이터 셋팅
        self.data_setting()

        time.sleep(10)

        # 데이터 저장
        self.df.to_excel('patents_data.xlsx', index=False, encoding='utf-8-sig')
        self.df.to_csv('patents_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
try :
    kipris = Kipris()

    kipris.start()
except Exception as e:
    print(e)

